In [1]:
import requests
import pandas as pd

# URL of the API endpoint
url = 'https://api.citybik.es/v2/networks/bixi-montreal'

# GET request to the API
response = requests.get(url)

In [2]:
# Check if the request was successful (status code 200)

if response.status_code != 200:
    print("Request failed. Status code:", response.status_code)
else:
    data = response.json()
    
    # Extract station details
    stations = data['network']['stations']

    # Create lists to store extracted details
    name = []
    latitudes = []
    longitudes = []

    # Iterate through each station and extract details
    for station in stations:
        name.append(station['name'])
        latitudes.append(station['latitude'])
        longitudes.append(station['longitude'])

In [3]:
# Create a DataFrame with the extracted details
df = pd.DataFrame({
    'Station Name': name,
    'Latitude': latitudes,
    'Longitude': longitudes
})

In [4]:
print(df)

                                          Station Name   Latitude  Longitude
0        Gare d'autocars de Montréal (Berri / Ontario)  45.516926 -73.564257
1                                   Marché Maisonneuve  45.553219 -73.539782
2                                  Rachel / de Brébeuf  45.526890 -73.572640
3        Bibliothèque d'Ahuntsic (Lajeunesse / Fleury)  45.553400 -73.662255
4    Cité des Arts du Cirque (Paul Boutet / des Reg...  45.559842 -73.615447
..                                                 ...        ...        ...
156                         du Mont-Royal / de Brébeuf  45.529337 -73.577953
157                                    K5 Test Station  45.530298 -73.601398
158                              Ste-Catherine / Union  45.503704 -73.569415
159  Place du Canada (Peel / des Canadiens-de-Montr...  45.497796 -73.568251
160                                   Aylwin / Ontario  45.544290 -73.545232

[161 rows x 3 columns]


In [5]:
df.to_csv('bike_stations.csv',index=False)

In [6]:
import os
import time
import json
import csv

PLACES_API_KEY = os.environ['PLACES_API_KEY']

In [11]:
def find_metro_stations(api_key, latitude, longitude, radius=1000):
    endpoint = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    location = f"{latitude},{longitude}"

    params = {
        'location': location,
        'radius': radius,
        'types': 'subway_station',
        'key': api_key,
    }

    response = requests.get(endpoint, params=params)
    results = response.json()

    if response.status_code == 200 and results['status'] == 'OK':
        return len(results['results'])
    else:
        print(f"Error: {results['status']}")
        return None

def process_csv(api_key, csv_file):
    with open('output.csv', 'r') as file:
        reader = csv.DictReader(file)

        for row in reader:
            latitude = float(row['Latitude'])  
            longitude = float(row['Longitude'])  

            metro_station_count = find_metro_stations(api_key, latitude, longitude)

            if metro_station_count is not None:
                print(f"For location ({latitude}, {longitude}), number of metro stations within 1km radius: {metro_station_count}")
            else:
                print(f"Unable to fetch data for location ({latitude}, {longitude})")

api_key = PLACES_API_KEY
csv_file_path = 'metro_stations.csv'  # Update with the path to your CSV file

process_csv(api_key, csv_file_path)

For location (45.5361062, -73.6135235), number of metro stations within 1km radius: 3
For location (45.498627, -73.5709641), number of metro stations within 1km radius: 6
For location (45.501355, -73.570814), number of metro stations within 1km radius: 8
For location (45.58318389999999, -73.54243559999999), number of metro stations within 1km radius: 3
For location (45.5085893, -73.5721164), number of metro stations within 1km radius: 4
For location (45.495379, -73.57928), number of metro stations within 1km radius: 5
For location (45.4948234, -73.57765429999999), number of metro stations within 1km radius: 6
For location (45.5257478, -73.58154979999999), number of metro stations within 1km radius: 2
Error: ZERO_RESULTS
Unable to fetch data for location (45.4529508, -73.6458851)
For location (45.5026725, -73.57171389999999), number of metro stations within 1km radius: 8
For location (45.4756456, -73.60702669999999), number of metro stations within 1km radius: 1
For location (45.5077116

In [20]:
def find_bus_stations(api_key, latitude, longitude, radius=1000):
    endpoint = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    location = f"{latitude},{longitude}"

    params = {
        'location': location,
        'radius': radius,
        'types': 'bus_station',
        'key': api_key,
    }

    bus_station_count = 0
    next_page_token = None

    while True:
        if next_page_token:
            params['pagetoken'] = next_page_token

        response = requests.get(endpoint, params=params)
        results = response.json()

        if response.status_code == 200 and results['status'] == 'OK':
            bus_station_count += len(results['results'])

            # Check if there are more pages
            next_page_token = results.get('next_page_token')

            # Wait for a short time before making the next request
            time.sleep(2)

            # Break the loop if there are no more pages
            if not next_page_token:
                break
        else:
            print(f"Error: {results['status']}")
            return None

    return bus_station_count

def process_csv2(api_key, input_csv2, output_csv2):
    with open(input_csv2, 'r') as infile, open(output_csv2, 'w', newline='') as outfile:
        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames + ['Bus_Stations_Count']

        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        for row in reader:
            latitude = float(row['Latitude'])  
            longitude = float(row['Longitude'])

            bus_station_count = find_bus_stations(api_key, latitude, longitude)

            if bus_station_count is not None:
                row['Bus_Stations_Count'] = bus_station_count
                print(f"For location ({latitude}, {longitude}), number of bus stations within 1km radius: {bus_station_count}")
            else:
                print(f"Unable to fetch data for location ({latitude}, {longitude})")

            writer.writerow(row)

api_key = PLACES_API_KEY
input_csv_path = 'output.csv'  # Update with the path to your input CSV file
output_csv_path = 'bus_stations.csv'  # Update with the desired path for the output CSV file

process_csv2(api_key, input_csv_path, output_csv_path)

For location (45.5361062, -73.6135235), number of bus stations within 1km radius: 18
For location (45.498627, -73.5709641), number of bus stations within 1km radius: 31
For location (45.501355, -73.570814), number of bus stations within 1km radius: 33
For location (45.58318389999999, -73.54243559999999), number of bus stations within 1km radius: 18
For location (45.5085893, -73.5721164), number of bus stations within 1km radius: 22
For location (45.495379, -73.57928), number of bus stations within 1km radius: 28
For location (45.4948234, -73.57765429999999), number of bus stations within 1km radius: 30
For location (45.5257478, -73.58154979999999), number of bus stations within 1km radius: 12
For location (45.4529508, -73.6458851), number of bus stations within 1km radius: 1
For location (45.5026725, -73.57171389999999), number of bus stations within 1km radius: 33
For location (45.4756456, -73.60702669999999), number of bus stations within 1km radius: 8
For location (45.5077116, -73.5